In [1]:
import pandas as pd
import sqlite3

In [2]:
connection = sqlite3.connect('../german_credit.db')
def select(query):
    return pd.read_sql_query(query, connection)

In [32]:
query = r'''
    SELECT * FROM german_credit
    LIMIT 10
'''
select(query)

,age,sex,job,housing,saving_accounts,checking_account,credit_amount,duration,purpose,default,contract_dt,client_id
0,33,male,2,own,None,None,3074,9,radio/TV,0,2008-06-29 18:52:00,210
1,43,male,1,own,little,little,1344,12,car,0,2007-05-20 18:30:19,929
2,52,male,2,own,quite rich,None,936,9,education,0,2008-04-27 08:23:07,200
3,35,female,3,own,little,None,1393,11,car,0,2007-05-06 10:58:22,45
4,28,male,2,own,little,None,776,12,radio/TV,0,2007-07-21 13:22:14,358
5,27,female,2,own,little,moderate,1295,18,furniture/equipment,0,2008-06-18 04:10:05,86
6,29,female,2,own,little,moderate,959,9,furniture/equipment,1,2007-05-21 14:32:00,351
7,42,female,2,rent,rich,rich,409,12,radio/TV,0,2008-06-05 05:31:13,27
8,26,male,2,own,little,little,4370,42,radio/TV,1,2007-11-29 00:20:44,639
9,55,male,2,free,rich,None,1555,12,repairs,1,2007-09-11 20:27:28,754


In [41]:
query = r'''
    SELECT 
        gc.credit_amount AS credit_amount,
        AVG(CASE WHEN gc.credit_amount > 1000 THEN 1 ELSE 0 END) AS greater_1000
    FROM german_credit AS gc
    --LIMIT 10
'''
select(query)

,credit_amount,greater_1000
0,3074,0.884


In [7]:
query = r'''
    DROP TABLE IF EXISTS german_credit_gt_2000;
    CREATE TABLE IF NOT EXISTS german_credit_gt_2000 AS 
    SELECT * 
    FROM german_credit AS gc
    WHERE credit_amount > 2000
'''
cursor = connection.cursor()
cursor.executescript(query)

In [20]:
query = r'''
    SELECT * 
    FROM german_credit_gt_2000 AS gc
'''
(
    pd.read_sql_query(query, connection)
    .assign(housing = lambda x: (x['housing'] == 'own').astype('int64'))
    ['housing'].mean() * 100

).round(2)

68.49

In [25]:
query = r'''
    WITH summary AS (
    SELECT * 
    FROM german_credit AS gc
    UNION ALL
    SELECT * 
    FROM german_credit_gt_2000 AS gc_2000
    ), dup_del_table AS (
    SELECT *, ROW_NUMBER() OVER w AS dup_del
    FROM summary AS sm
    WINDOW w AS (PARTITION BY contract_dt)
    )
    SELECT * 
    FROM dup_del_table 
    WHERE dup_del = 1
'''
select(query)

,age,sex,job,housing,saving_accounts,checking_account,credit_amount,duration,purpose,default,contract_dt,client_id,dup_del
0,20,male,2,own,None,little,2996,24,furniture/equipment,1,2007-05-01 07:17:29,495,1
1,49,female,2,own,little,moderate,1092,12,radio/TV,0,2007-05-01 08:41:46,288,1
2,30,female,3,own,little,moderate,4795,36,radio/TV,0,2007-05-01 12:42:45,141,1
3,39,female,1,own,None,moderate,932,6,education,0,2007-05-01 20:28:44,215,1
4,31,male,2,own,little,None,2775,18,car,1,2007-05-01 23:14:02,598,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,58,male,2,own,None,None,1867,30,radio/TV,0,2008-06-28 11:49:14,928,1
996,28,female,2,own,little,None,1382,6,radio/TV,0,2008-06-28 12:33:38,196,1
997,44,male,2,free,little,None,5507,24,repairs,0,2008-06-28 14:08:06,520,1
998,39,male,3,own,little,moderate,11938,24,vacation/others,1,2008-06-28 23:47:51,105,1


In [31]:
query = r'''
    UPDATE german_credit
    SET sex = UPPER(SUBSTR(sex, 1, 1))
'''
select(query)

TypeError: 'NoneType' object is not iterable

In [33]:
query = r'''
    UPDATE german_credit
    SET sex = (CASE WHEN sex = 'M' THEN 'male' ELSE 'female' END)
'''
select(query)

TypeError: 'NoneType' object is not iterable

In [34]:
query = r'''
    SELECT * 
    FROM german_credit AS gc
    LIMIT 10
'''
select(query)

,age,sex,job,housing,saving_accounts,checking_account,credit_amount,duration,purpose,default,contract_dt,client_id
0,33,male,2,own,None,None,3074,9,radio/TV,0,2008-06-29 18:52:00,210
1,43,male,1,own,little,little,1344,12,car,0,2007-05-20 18:30:19,929
2,52,male,2,own,quite rich,None,936,9,education,0,2008-04-27 08:23:07,200
3,35,female,3,own,little,None,1393,11,car,0,2007-05-06 10:58:22,45
4,28,male,2,own,little,None,776,12,radio/TV,0,2007-07-21 13:22:14,358
5,27,female,2,own,little,moderate,1295,18,furniture/equipment,0,2008-06-18 04:10:05,86
6,29,female,2,own,little,moderate,959,9,furniture/equipment,1,2007-05-21 14:32:00,351
7,42,female,2,rent,rich,rich,409,12,radio/TV,0,2008-06-05 05:31:13,27
8,26,male,2,own,little,little,4370,42,radio/TV,1,2007-11-29 00:20:44,639
9,55,male,2,free,rich,None,1555,12,repairs,1,2007-09-11 20:27:28,754
